In [ ]:
DATA_PATH = '../input/shopee-product-matching/'
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

train = pd.read_csv(DATA_PATH + 'train.csv')
train['image'] = DATA_PATH + 'train_images/' + train['image']
train.head()

In [ ]:
def findByLabel(label_group_idx, showImgNumber=10, color=None):
    train_a = train[train.label_group == label_group_idx].reset_index()
  
    fig, ax = plt.subplots(2, 5, figsize=(20, 10))
    for i in range(2):
        for j in range(5):
            if showImgNumber == 0: break
            ax[i, j].imshow(cv2.imread(train_a.image[showImgNumber-1], color))
            showImgNumber-=1
        
    return train_a

In [ ]:
findByLabel(1163569239)
plt.show()

In [ ]:
findByLabel(1163569239, color=0)
plt.show()

In [ ]:
findByLabel(3627744656)
plt.show()

In [ ]:
findByLabel(3627744656, color=0)
plt.show()

It turns out that setting the pic to single color channel (i.e. color=0) may not be a good option as it turns all the pics into similar color.

In [ ]:
label_train = train[train.label_group==1163569239].reset_index()
label_train

In [ ]:
img1 = cv2.imread(label_train.image[4])
color = ('r', 'g', 'b')
for i, color in enumerate(color):
    hist = cv2.calcHist([img1], [i], None, [256], [0, 256])
    plt.title('img1')
    plt.xlabel('Bins')
    plt.ylabel('Frequency')
    plt.plot(hist, color=color, label=color)
plt.legend()
plt.show()

In [ ]:
colors = ('r', 'g', 'b')
fig, ax = plt.subplots(10, 2, figsize=(8, 20))
for i in range(10):
    img = cv2.imread(label_train.image[i])
    ax[i, 0].imshow(img)
    for k, color in enumerate(colors):
        hist = cv2.calcHist([img], [k], None, [256], [0, 256])
        ax[i, 1].plot(hist, color=color, label=color)
        ax[i, 1].legend()
plt.show()

It turns out that product within the same label group have different histogram with respect to RGB. Hist method may not be a good way to separate them out.

In [ ]:
train_label = train.groupby('label_group').posting_id.count().sort_values(ascending=True)
train_label

In [ ]:
pd.cut(train_label, bins=[0, 10, 20, 30, 40, 51]).value_counts()

In [ ]:
def creatImages(img_path):
    '''
    input: img_path
    output: list. 4 pics
    function: 1 pic in, 4 pics out. Generate more pics in label_group
    '''
    img = cv2.imread(img_path)
    row, col = int(img.shape[0]), int(img.shape[1])
    img1 = img[0:int(2*row/3), 0:int(2*col/3)]
    img2 = img[0:int(2*row/3), int(1*col/3):col]
    img3 = img[int(1*row/3):row, 0:int(2*col/3)]
    img4 = img[int(1*row/3):row, int(1*col/3):col]
    images = [img1, img2, img3, img4]
    return images

In [ ]:
images = creatImages(train.image[2123])
for i in range(len(images)):
    plt.subplot(2, 2, i+1)
    plt.imshow(images[i])
plt.show()

Here I offerred one way to generate more pics from dataset, which could be useful for the further training. Let's see.